In [ ]:
"""
Created on 26.09.2023
"""

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def FrankeFunction(x, y):
    term1 = 0.75 * np.exp(-(0.25 * (9 * x - 2)**2) - 0.25 * ((9 * y - 2)**2))
    term2 = 0.75 * np.exp(-((9 * x + 1)**2) / 49.0 - 0.1 * (9 * y + 1))
    term3 = 0.5 * np.exp(-(9 * x - 7)**2 / 4.0 - 0.25 * ((9 * y - 3)**2))
    term4 = -0.2 * np.exp(-(9 * x - 4)**2 - (9 * y - 7)**2)
    return term1 + term2 + term3 + term4


def create_X(x, y, n):
    """Returns the design matrix X from coordinates x and y with n polynomial degrees."""
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n + 1) * (n + 2) / 2)  # Number of elements in beta
    X = np.ones((N, l))

    for i in range(1, n + 1):
        q = int(i * (i + 1) / 2)
        for k in range(i + 1):
            X[:, q + k] = (x**(i - k)) * (y**k)

    return X


def MSE(y, y_tilde):
    """Returns the mean squared error of the two arrays."""
    return np.mean((y - y_tilde)**2)


def R2_score(y, y_tilde):
    """Returns the R2 score of the two arrays."""
    return 1 - np.sum((y - y_tilde)**2) / np.sum((y - np.mean(y))**2)


# PARAMETERS
n = 1000  # number of data points
maxdegree = 6  # max polynomial degree for plotting
lmbda_vals = np.asarray([0.0001, 0.001, 0.01, 0.1, 1])

# Figure output directory
RESULTS_DIR = Path("../results").resolve()
FIGURES_DIR = RESULTS_DIR / "figures"

# Create them if they dont exist
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir()

if not FIGURES_DIR.exists():
    FIGURES_DIR.mkdir()

# Create data set
np.random.seed(2023)
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))

# Get franke function with noise
noise = np.random.normal(0, 0.1, x.shape)
z = FrankeFunction(x, y) + noise

scaler = StandardScaler()

# Identity matrix
I = np.identity(maxdegree)

degrees = np.arange(0, maxdegree, 1)
test_mse = np.empty(degrees.shape)
test_r2 = np.empty_like(test_mse)
beta_arr = []

# Iterate through all the different polynomial degrees
for degree in degrees:
    for lmbda in lmbda_vals:
        # Create design matrix X
        X = create_X(x, y, n=degree)  # intercept included

        # Split in training and test data up to the given degree
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)

        # Scale data
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Perform ridge regression
        beta = (np.linalg.pinv(X_train_scaled.T @ X_train_scaled + lmbda * I) @ X_train_scaled.T @ z_train)
        z_tilde = X_train_scaled @ beta
        z_predict = X_test_scaled @ beta

        # Store beta value for plotting
        beta_arr.append(beta)

        # Store the MSE and R2 values
        test_mse[degree] = MSE(z_test, z_predict)
        test_r2[degree] = R2_score(z_test, z_predict)